# 实现RNN

In [1]:
import json
import time
import random
import numpy as np
import pandas as pd
import math
import psutil
import gc
from collections import Counter
import warnings
warnings.filterwarnings('ignore')
import sys
# from sklearn.externals import joblib
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
import time
from torch.autograd import Variable
from glob import glob
from sys import getsizeof
import os
import torch.nn as nn
from torch.optim import lr_scheduler
from torch import optim
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid
import shutil
from torchvision import transforms
from torchvision import models
from torchtext import data, datasets
from nltk import ngrams
from torchtext.vocab import GloVe, Vectors
from collections import defaultdict
data_path = '/kaggle/input/spooky-author-identification/'
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from nltk import word_tokenize
from nltk.corpus import stopwords
import nltk
stop_words = stopwords.words('english')
from torch.nn import utils as nn_utils
print (torch.cuda.is_available())

data_path = r'D:\kaggle\data\spooky-author-identification\a'[: -1]
data_path_inv = r'D:\kaggle\data\spooky-author-identification\a'[: -1]
data_path_word_vector = r'D:\kaggle\data\word_vector\a'[: -1]

True


In [2]:
def read_data():
    df_train = pd.read_csv(data_path_inv + r'train/train.csv')
    df_test = pd.read_csv(data_path + 'test/test.csv')
    df_sub = pd.read_csv(data_path + 'sample_submission/sample_submission.csv')
    return df_train, df_test, df_sub
df_train, df_test, df_sub = read_data()
print (df_train.shape, df_test.shape, df_sub.shape)
print (df_train.head(3), '\n\n', df_test.head(3), '\n\n', df_sub.head(3), )

(19579, 3) (8392, 2) (8392, 4)
        id                                               text author
0  id26305  This process, however, afforded me no means of...    EAP
1  id17569  It never once occurred to me that the fumbling...    HPL
2  id11008  In his left hand was a gold snuff box, from wh...    EAP 

         id                                               text
0  id02310  Still, as I urged our leaving Ireland with suc...
1  id24541  If a fire wanted fanning, it could readily be ...
2  id00134  And when they had broken down the frail door t... 

         id       EAP       HPL       MWS
0  id02310  0.403494  0.287808  0.308698
1  id24541  0.403494  0.287808  0.308698
2  id00134  0.403494  0.287808  0.308698


In [4]:
# 处理数据
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(df_train.author.values)
xtrain, xvalid, ytrain, yvalid = train_test_split(df_train.text.values, y, 
                                                  stratify = y, 
                                                  random_state = 2020, 
                                                  test_size = 0.1, shuffle = True)
xtest = df_test.text.values
print (xtrain.shape)
print (xvalid.shape)
print (xtest.shape)

(17621,)
(1958,)
(8392,)


In [6]:
# load the GloVe vectors in a dictionary:
embeddings_word_set = set()
def sent2vec_add_word(s):
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    for w in words:
        embeddings_word_set.add(str.encode(w))
    return 0

temp = [sent2vec_add_word(x) for x in xtrain]
del temp
gc.collect()
temp = [sent2vec_add_word(x) for x in xvalid]
del temp
gc.collect()
temp = [sent2vec_add_word(x) for x in xtest]
del temp
gc.collect()

# %% [code]
# load the GloVe vectors in a dictionary:
embeddings_index = {}
f = open(data_path_word_vector + 'glove.840B.300d.txt', 'rb')
index = 0
pre_time = time.time()
for line in f: # tqdm(f):
    index += 1
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    if word in embeddings_word_set:
        embeddings_index[word] = coefs
    if index % 500000 == 0:
        print ('index: {:}, time:{:}'.format(index, time.time() - pre_time))
f.close()
print('Found %s word vectors.' % len(embeddings_index))

index: 500000, time:40.3858962059021
index: 1000000, time:80.61405372619629
index: 1500000, time:121.12958931922913
index: 2000000, time:161.22476959228516
Found 25931 word vectors.


In [7]:
def sent2vec(s):
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            torch_tmp = list(embeddings_index[str.encode(w)])
            M.append(torch_tmp)
        except:
            continue
    if len(M) == 0:
        M.append([0] * 300)
    return M

xtrain_glove = [sent2vec(x) for x in xtrain]
xvalid_glove = [sent2vec(x) for x in xvalid]
xtest_glove = [sent2vec(x) for x in xtest]

In [16]:
xtrain_torch = torch.zeros((10000, 50, 300)).long().cuda()
print (xtrain_torch.size())

RuntimeError: CUDA out of memory. Tried to allocate 1.12 GiB (GPU 0; 6.00 GiB total capacity; 4.16 GiB already allocated; 604.89 MiB free; 817.00 KiB cached)

In [10]:
word_vector_size = len(xtrain_glove[0][0])
label_size = 3
print ('word_vector_size: {:}, label_size: {:}'.format(word_vector_size, label_size))

xtrain_lengths = torch.LongTensor([len(x) for x in xtrain_glove]).cuda()
print (xtrain_lengths.max())
max_length = 100
xtrain_torch = torch.zeros((len(xtrain_glove), max_length, word_vector_size)).long().cuda()
for idx in range(len(xtrain_glove)):
    seqlen = min(int(xtrain_lengths[idx].cpu().numpy()), max_length)
    xtrain_torch[idx, :seqlen] = torch.LongTensor(np.array(xtrain_glove[idx])[: seqlen, :])

print (type(xtrain_torch), xtrain_torch.size())
xtrain_lengths, seq_idx = xtrain_lengths.sort(0, descending = False)
xtrain_torch = xtrain_torch[seq_idx]

word_vector_size: 300, label_size: 3
tensor(252, device='cuda:0')


RuntimeError: CUDA out of memory. Tried to allocate 3.94 GiB (GPU 0; 6.00 GiB total capacity; 3.94 GiB already allocated; 833.89 MiB free; 817.00 KiB cached)

In [ ]:
print (type(xtrain_glove), xtrain_glove.shape)
len_min = 10000000
len_max = 0
for i in range(len(xtrain_glove)):
    len_min = min(len_min, len(xtrain_glove[i]))
    len_max = max(len_max, len(xtrain_glove[i]))
    # print (type(xtrain_glove[i]), len(xtrain_glove[i]))
print (len_min, len_max)
print (len(xtrain_glove[0][0]))
print (len(xtrain_glove[1][-1]))
print (type(xtrain_glove[0]))
print (type(xtrain_glove[0][0]))

xtrain_glove = np.array(xtrain_glove)
print (xtrain_glove.shape)
print (type(xtrain_glove[0, :]))
print (type(xtrain_glove[0][0]))

In [ ]:
print (ytrain[: 20])
print (type(xtrain_glove), xtrain_glove.shape)
xtrain_tmp = [[xtrain_glove[i], ytrain[i]] for i in range(len(xtrain_glove))]
xtrain_tmp = np.array(sorted(xtrain_tmp, key = lambda x: len(x[0]), reverse = True))
xtrain_sort_len = xtrain_tmp[: , 0]
ytrain = xtrain_tmp[: , 1]
print (type(ytrain))
print (ytrain[: 20])
for i in range(10):
    print (i, len(xtrain_sort_len[i]))

In [ ]:
print (type(xtrain_glove), xtrain_glove.shape)
xtrain_torch = torch.from_numpy(xtrain_glove)
print (xtrain_torch.size())

In [ ]:
embed_input_x_packed = pack_padded_sequence(xtrain_glove, xtrain_len, batch_first=True)
encoder_outputs_packed, (h_last, c_last) = self.lstm(embed_input_x_packed)
encoder_outputs, _ = pad_packed_sequence(encoder_outputs_packed, batch_first=True)

# demo

In [ ]:
### 定义模型
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space，相当于一个全连接层
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))  # 把三维张量转化为二级张量
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

### 模型训练及预测
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()    # 调用时形式为：预测值(N*C),label(N)。其中N为序列中word数，C为label的类别数
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)  # 此处的inputs只能是一个sequence
    print(tag_scores)

for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)    #一个sequence对应的词性标注list

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print("[result]tag_scores={}".format(tag_scores))